<font color = "red"> Binana close contacts! 

#### Overview 

| Task                                      | Description                                                   |
|-------------------------------------------|---------------------------------------------------------------|
| Run Binana                                | Execute Binana for all PDBs and poses                         |
| Read logfile                              | Create a dictionary with information from log files           |
| Getting dict info                         | - Close Contacts [OK] <br> - Hydrogen Bonds                  |
| Extract atom properties (RDkit)           | - Atom Index <br> - Name <br> - Mass <br> - Charge <br> - Element Name <br> - Hybridization <br> - Num Hydrogens <br> - Formal Charge <br> - Unpaired Electron <br> - In Aromatic Substructure |
| Preparing data for graph                  | - protgraph_dict[pdb][pose] = {<br>&nbsp;&nbsp;&nbsp;&nbsp;'x_s': x_s (ligand properties),<br>&nbsp;&nbsp;&nbsp;&nbsp;'x_t': x_t (receptor properties),<br>&nbsp;&nbsp;&nbsp;&nbsp;'edge_index': edge_index (Matrix [2, number of connections]),<br>&nbsp;&nbsp;&nbsp;&nbsp;'edge_attr': edge_attr (Vector [distances])<br>} |
| Protgraph_dict                            | --> BipartiteData data  --> Dataset list  --> DataLoader  --> Graph Attention Model |


* Data spliting methods

### Libraries 

In [1]:
from tqdm import tqdm
import pickle
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import subprocess
import numpy as np
import torch
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import os
from torch_geometric.loader import DataLoader
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from collections import Counter
from torch_geometric.data import Data, Dataset
import json
import os
import pandas as pd
import pandas as pd
from rdkit import Chem
import torch
import numpy as np
from torch.utils.data import Dataset
import pickle
from collections import defaultdict
from pathlib import Path

/home/lbcb02/anaconda3/envs/master_37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run Config.ipynb

In [3]:
config = Config()

In [4]:
pdbs = os.listdir(config.coreset)
nposes = config.docking_params['nposes']
batch_size  = config.model_args['batch_size']
path_scripts = Path(os.getcwd())
num_epochs = config.model_args['epochs']
num_folds = config.model_args['nfolds']
patience = config.model_args['patience']  # Set your desired patience value
binana_executable = config.executables['binana']

In [5]:
len(pdbs)

40

### Running Binana
<font color = "red"> Só precisa rodar uma vez!

In [6]:
%%time
def binana(pdb, config):
    os.makedirs(config['output_folder'], exist_ok=True)  # Create the directory path if it doesn't exist
    binana_exec_cmd = f"python3 {binana_executable} -receptor {config['receptor_path']} -ligand {config['ligand_path']} -output_dir {config['output_folder']} > errors.txt"
    os.system(binana_exec_cmd)


# Combined loop for both versions
for pdb in tqdm(pdbs, desc="Processing PDBS"):

    config_version_2 = {
    'ligand_path': f"{config.coreset}/{pdb}/ligand.pdbqt",
    'output_folder': f"{config.coreset}/{pdb}/binana/binana_{pdb}",
    'receptor_path': f"{config.coreset}/{pdb}/receptor.pdbqt"
}
    
    for pose in range(1, nposes + 1):
        # Configuration dictionaries for each version
        config_version_1 = {
            'ligand_path':f"{config.coreset}/{pdb}/results/pose_{pose}.pdbqt",
            'output_folder': f"{config.coreset}/{pdb}/binana/binana_{pdb}_pose_{pose}",
            'receptor_path': f"{config.coreset}/{pdb}/receptor.pdbqt"
        }
        try:
            binana(pdb, config=config_version_1)
        except ValueError as e:
            print(e)
            pass
    binana(pdb, config=config_version_2)

Processing PDBS:  40%|██████████▍               | 16/40 [01:46<02:45,  6.89s/it]Traceback (most recent call last):
  File "/home/lbcb02/Workspace/BindRanker/BindRanker/Datafactory/../executables/binana-2.1/python/run_binana.py", line 13, in <module>
    start()
  File "/home/lbcb02/Workspace/BindRanker/BindRanker/Datafactory/../executables/binana-2.1/python/run_binana.py", line 8, in start
    binana.run()
  File "/home/lbcb02/Workspace/BindRanker/BindRanker/executables/binana-2.1/python/binana/__init__.py", line 97, in run
    _start._get_all_interactions(cmd_params)
  File "/home/lbcb02/Workspace/BindRanker/BindRanker/executables/binana-2.1/python/binana/_start.py", line 48, in _get_all_interactions
    ligand, receptor = from_files(
                       ^^^^^^^^^^^
  File "/home/lbcb02/Workspace/BindRanker/BindRanker/executables/binana-2.1/python/binana/load_ligand_receptor/__init__.py", line 98, in from_files
    ligand.load_pdb_file(ligand_filename)
  File "/home/lbcb02/Workspac

CPU times: user 135 ms, sys: 95.7 ms, total: 231 ms
Wall time: 4min 41s


### Read logfile

In [7]:
%%time
data_dict_all_logs_exp = defaultdict(dict)
data_dict_all_logs_docked = defaultdict(dict)
def read_logfile(pdb, pose=None):
    # Define the common part of the file path
    common_path = f"{config.coreset}/{pdb}/binana/binana_{pdb}"
    
    # Adjust the file path based on whether the pose parameter is provided
    path = common_path + (f"_pose_{pose}" if pose is not None else "") + "/log.txt"

     
    try: # Read the text file using Path
        with Path(path).open('r') as file:
            text = file.read()
    
    
        # Extract the content after "JSON Output:"
        json_text = text.split("JSON Output:")[1].strip()
        
        # Convert the JSON text to a dictionary
        data_dict = json.loads(json_text)
        
        # Store the data dictionary for the pose if pose is provided
        if pose is not None:
            data_dict_all_logs_docked[pdb][pose] = data_dict
        else:
            # Store the data dictionary for the pose (from the first code)
            data_dict_all_logs_exp[pdb] = data_dict

    except:
        print("Error when opening file ", path)
    
    return data_dict_all_logs_exp, data_dict_all_logs_docked

# Example usage: provide pose parameter for the second code, leave it empty for the first code
for pdb in pdbs:
    read_logfile(pdb)  # Run logic from the first code
    for pose in range(1, nposes + 1):
        read_logfile(pdb, pose)  # Run logic from the second code

Error when opening file  ../coreset/3f3c/binana/binana_3f3c_pose_7/log.txt
Error when opening file  ../coreset/3f3c/binana/binana_3f3c_pose_8/log.txt
Error when opening file  ../coreset/3f3c/binana/binana_3f3c_pose_9/log.txt
Error when opening file  ../coreset/3f3c/binana/binana_3f3c_pose_10/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_3/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_4/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_5/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_6/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_7/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_8/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_9/log.txt
Error when opening file  ../coreset/2yge/binana/binana_2yge_pose_10/log.txt
Error when opening file  ../coreset/2yki/binana/binana_2yki_pose_5/log.txt
Error when opening file

In [8]:
data_dict_all_logs_docked.get('2qbq')

{1: {'activeSiteFlexibility': {'BACKBONE_ALPHA': 8,
   'BACKBONE_OTHER': 7,
   'SIDECHAIN_ALPHA': 45,
   'SIDECHAIN_BETA': 3,
   'SIDECHAIN_OTHER': 75},
  'cationPiInteractions': [],
  'closeContacts': [{'ligandAtoms': [{'atomIndex': 4,
      'atomName': 'C12',
      'chain': 'd',
      'resID': 0,
      'resName': 'MOL'}],
    'metrics': {'distance': 3.9111344901447693},
    'receptorAtoms': [{'atomIndex': 509,
      'atomName': 'CG2',
      'chain': 'A',
      'resID': 49,
      'resName': 'VAL'}]},
   {'ligandAtoms': [{'atomIndex': 5,
      'atomName': 'C13',
      'chain': 'd',
      'resID': 0,
      'resName': 'MOL'}],
    'metrics': {'distance': 3.956145725324084},
    'receptorAtoms': [{'atomIndex': 467,
      'atomName': 'CB',
      'chain': 'A',
      'resID': 46,
      'resName': 'TYR'}]},
   {'ligandAtoms': [{'atomIndex': 5,
      'atomName': 'C13',
      'chain': 'd',
      'resID': 0,
      'resName': 'MOL'}],
    'metrics': {'distance': 3.552690248248501},
    'receptorA

### Getting dict info 

In [9]:
%%time
def flatten_data(df, pdb, pose = False):
    flat_data = []
    if pose == False:
        data = df[pdb]['closeContacts']
    else:
        data = df[pdb][pose]['closeContacts']
    
    for entry in data:
        flat_entry = {
            'ligand_atom_index': entry['ligandAtoms'][0]['atomIndex'],
            'ligand_atom_name': entry['ligandAtoms'][0]['atomName'],
            'ligand_chain': entry['ligandAtoms'][0]['chain'],
            'ligand_resID': entry['ligandAtoms'][0]['resID'],
            'ligand_resName': entry['ligandAtoms'][0]['resName'],
            #'angle': entry['metrics']['angle'],
            'distance': entry['metrics']['distance'],
            'receptor_atom_index': entry['receptorAtoms'][0]['atomIndex'],
            'receptor_atom_name': entry['receptorAtoms'][0]['atomName'],
            'receptor_chain': entry['receptorAtoms'][0]['chain'],
            'receptor_resID': entry['receptorAtoms'][0]['resID'],
            'receptor_resName': entry['receptorAtoms'][0]['resName'],
        }
        flat_data.append(flat_entry)

    return pd.DataFrame(flat_data)


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [10]:
#data_dict_all_logs_docked

In [11]:
#data_dict_all_logs_docked.get("2qbq")

In [12]:
%%time
data_frames_exp = [
    flatten_data(data_dict_all_logs_exp, pdb).assign(pdb=pdb)
    for pdb in pdbs
]
data_frame_exp = pd.concat(data_frames_exp, axis=0)

#data_frames_docked = [flatten_data(data_dict_all_logs_docked, pdb, pose).assign(pdb=pdb, pose=str(pose))  for pdb in pdbs  for pose in range(1, nposes)]
#data_frame_docked = pd.concat(data_frames_docked, axis=0)

CPU times: user 141 ms, sys: 8.48 ms, total: 149 ms
Wall time: 170 ms


In [13]:
%%time
# Create a list of DataFrames with the specified conditions
data_frames_docked = [
    flatten_data(data_dict_all_logs_docked, pdb, pose).assign(pdb=pdb, pose=str(pose))
    for pdb in pdbs
    for pose in range(1, nposes)
    if pose in data_dict_all_logs_docked.get(pdb, {})
]

# Concatenate the DataFrames
data_frame_docked = pd.concat(data_frames_docked, axis=0)

CPU times: user 1.57 s, sys: 12.5 ms, total: 1.59 s
Wall time: 1.61 s


In [14]:
df = data_frame_docked.copy()

In [15]:
df[df['pdb'] == '1owh'].head()

,ligand_atom_index,ligand_atom_name,ligand_chain,ligand_resID,ligand_resName,distance,receptor_atom_index,receptor_atom_name,receptor_chain,receptor_resID,receptor_resName,pdb,pose
0,1.0,C12,d,0.0,MOL,3.670637,1884.0,O,A,206.0,SER,1owh,1
1,1.0,C12,d,0.0,MOL,3.995045,2130.0,O,A,234.0,GLY,1owh,1
2,2.0,C13,d,0.0,MOL,3.995668,2109.0,H,A,232.0,GLY,1owh,1
3,2.0,C13,d,0.0,MOL,3.351076,2130.0,O,A,234.0,GLY,1owh,1
4,3.0,C3,d,0.0,MOL,3.905747,1897.0,N,A,208.0,GLN,1owh,1


In [16]:
df['pdb'].nunique()

40

#### List of indexes  

- Create a dictionary with the **indices** of close-contact atoms.

- This is necessary so that we know from which atoms we should **obtain the properties.**

- Dictionaries for experimental data and for poses obtained through docking, for the comparison **metric of native contacts.**

<font color = "red"> Why i used process_ids to sort the values 

In [17]:
data_frame_docked[data_frame_docked['pdb']== '1owh'].head()

,ligand_atom_index,ligand_atom_name,ligand_chain,ligand_resID,ligand_resName,distance,receptor_atom_index,receptor_atom_name,receptor_chain,receptor_resID,receptor_resName,pdb,pose
0,1.0,C12,d,0.0,MOL,3.670637,1884.0,O,A,206.0,SER,1owh,1
1,1.0,C12,d,0.0,MOL,3.995045,2130.0,O,A,234.0,GLY,1owh,1
2,2.0,C13,d,0.0,MOL,3.995668,2109.0,H,A,232.0,GLY,1owh,1
3,2.0,C13,d,0.0,MOL,3.351076,2130.0,O,A,234.0,GLY,1owh,1
4,3.0,C3,d,0.0,MOL,3.905747,1897.0,N,A,208.0,GLN,1owh,1


In [18]:
%%time
def process_ids(data_frame, pdb,  ligand_column, pose = False):
    if 'pose' in data_frame.columns:
        aux = list(data_frame[(data_frame['pdb'] == pdb) & (data_frame['pose'] == str(pose))][ligand_column])
    else:
        aux = list(data_frame[data_frame['pdb'] == pdb][ligand_column])
    try:
        aux = [int(val) for val in aux]
    except ValueError:
        pass
    return aux

def process_pdb_data(pdbs, data_frame, ligand_column):
    ids = {}
    for pdb in pdbs:
        if 'pose' in data_frame.columns:
            ids[pdb] = {}
            for pose in range(1, nposes + 1):
                ids[pdb][pose] = process_ids(data_frame, pdb,  ligand_column, pose)
        else:
            ids[pdb] = process_ids(data_frame, pdb, ligand_column)
    return ids

# index
ids_lig_with_poses = process_pdb_data(pdbs, data_frame_docked, 'ligand_atom_index')
ids_rec_with_poses  = process_pdb_data(pdbs, data_frame_docked, 'receptor_atom_index')
ids_lig_exp = process_pdb_data(pdbs, data_frame_exp, 'ligand_atom_index')
ids_rec_exp  = process_pdb_data(pdbs, data_frame_exp, 'receptor_atom_index')

## Names
#names_lig_with_poses = process_pdb_data(pdbs, data_frame_docked, 'ligand_atom_name')
#names_rec_with_poses  = process_pdb_data(pdbs, data_frame_docked, 'receptor_atom_name')
#names_lig_exp = process_pdb_data(pdbs, data_frame_exp, 'ligand_atom_name')
#names_rec_exp  = process_pdb_data(pdbs, data_frame_exp, 'receptor_atom_name')

CPU times: user 5.98 s, sys: 0 ns, total: 5.98 s
Wall time: 5.99 s


In [19]:
ids_rec_exp['1owh']

[1884,
 2130,
 1890,
 2130,
 2137,
 1897,
 1908,
 1897,
 1908,
 1898,
 1901,
 1908,
 1907,
 1929,
 1928,
 1929,
 1928,
 2078,
 2089,
 2090,
 2078,
 2091,
 2092,
 1905,
 1907,
 1929,
 1902,
 1903,
 1905,
 1906,
 1907,
 431,
 433,
 1929,
 431,
 432,
 433,
 1928,
 1929,
 431,
 433,
 431,
 1907,
 427,
 431,
 428,
 429,
 431,
 918,
 429,
 431,
 433,
 918,
 919,
 427,
 471,
 471,
 469,
 470,
 471,
 471,
 1877,
 1878,
 1879,
 1883,
 1884,
 2130,
 1877,
 1879,
 1884,
 2106,
 2129,
 2130,
 2133,
 2145,
 1879,
 2106,
 2107,
 2108,
 2129,
 2130,
 2132,
 2133,
 2137,
 1877,
 1878,
 1879,
 2129,
 2130,
 2133,
 2140,
 2145,
 2180,
 1877,
 1878,
 1879,
 1881,
 1883,
 1884,
 1885,
 1886,
 1887,
 1888,
 2198,
 1878,
 1881,
 1882,
 1883,
 1884,
 1885,
 1886,
 1887,
 1888,
 2198,
 1877,
 1878,
 1879,
 1881,
 1884,
 1886,
 1887,
 1888,
 2197,
 2198,
 2201]

In [20]:
#ids_rec_with_poses['1owh']

### Compare native contacts

In [21]:
%%time
def compare_graphs(pairs_graph1, pairs_graph2):
    """
    Compare two sets of graph pairs and calculate a similarity score.

    Parameters
    ----------
    pairs_graph1 : list
        List of pairs from the first graph.
    pairs_graph2 : list
        List of pairs from the second graph.

    Returns
    -------
    float
        Percentage similarity between the two graphs.
    """
    common_pairs = [pair for pair in pairs_graph1 if pair in pairs_graph2]
    print(len(common_pairs))
    similarity_score = (len(common_pairs) / len(pairs_graph2)) * 100
    return similarity_score

score = {}

for pdb in pdbs:
    score[pdb] = {}
    for pose in range(1, nposes + 1):
        pairs_graph1 = list(zip(ids_lig_with_poses[pdb][pose], ids_rec_with_poses[pdb][pose]))
        pairs_graph2 = list(zip(ids_lig_exp[pdb], ids_rec_exp[pdb]))
        score[pdb][pose] = compare_graphs(pairs_graph1, pairs_graph2)

import json 
with open(f"{config.data}/score.json", "w") as json_file:
    json.dump(score, json_file)

92
116
63
96
74
81
70
20
73
0
55
4
4
9
5
9
32
4
0
0
95
107
57
0
12
7
18
4
7
0
86
14
66
14
5
10
50
12
66
0
125
9
0
27
1
13
11
17
22
0
9
41
10
4
1
5
17
2
10
0
3
7
8
4
140
6
151
4
2
0
11
12
8
4
0
40
29
8
6
0
1
1
0
76
0
20
0
35
28
0
61
30
5
3
44
9
5
5
17
0
66
52
54
2
71
65
64
2
80
0
1
7
35
9
0
6
55
11
2
0
67
0
56
3
7
0
0
0
0
0
110
38
7
12
14
14
45
35
8
0
111
93
54
48
24
13
26
6
0
0
121
120
44
51
49
6
5
72
46
0
114
98
60
65
68
0
0
0
0
0
129
73
37
14
3
39
8
28
4
0
84
61
45
5
22
11
0
3
22
0
79
117
7
91
97
8
5
5
6
0
92
0
0
0
0
0
0
0
0
0
107
31
12
37
27
13
20
26
19
0
114
93
0
0
0
0
0
0
0
0
129
8
1
27
3
13
0
7
0
0
0
0
1
30
2
13
1
0
45
0
5
2
4
6
29
15
7
2
3
0
1
2
2
16
4
2
2
1
3
0
77
55
0
41
16
7
2
47
13
0
115
98
98
69
54
1
43
27
1
0
38
0
76
41
23
0
1
50
111
0
157
117
122
103
50
57
106
64
44
0
59
0
4
24
40
11
1
15
3
0
102
12
0
5
10
3
4
6
5
0
29
16
30
9
2
14
2
13
14
0
115
62
13
21
34
30
29
7
28
0
135
24
13
9
8
8
5
4
7
0
5
46
14
2
27
88
2
7
8
0
142
134
10
3
0
7
11
6
16
0
156
138
32
17
18
15
30
14
7


<font color = "yellow"> When i extrated the atom properties, i only extrated for  ids_3gv9_pose_1

Why am i taking this?

     * Because we only want properties from the important nodes

### Extract atom properties

In [51]:
def find_element_index(lst, element):
    """
    Locate the index of the desired element within the list. If the element is not found, return the index of the last item.
    """
    try:
        return lst.index(element)
    except ValueError:
        return len(lst) - 1

In [52]:
class AtomPropertiesExtractor:
    def __init__(self):
        #self.ligand_path = ligand_path
        self.columns = ["Atom_Index", "Name", "Mass", "Charge", "Element_Name", "Hybridization", "Num_Hydrogens", "Formal_Charge", "Unpaired_Electron", "In_Aromatic_Substructure"]
        self.allowable_features = {
            "possible_hybridization_list": ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "UNSPECIFIED"],
            "possible_atoms": ['C', 'S', 'N', 'O', 'H']
        }

    def extract_properties(self, atom_indices, properties_to_extract, path):
        atom_properties_list = []

        with open(path, "r") as pdb_file:
            pdb_data = pdb_file.read()

        molecule = Chem.MolFromPDBFile(path, removeHs=False)

        if molecule is not None:
            for atom_index in atom_indices:
                atom = molecule.GetAtomWithIdx(atom_index - 1)

                atom_properties = {
                    "Atom_Index": atom_index,
                    "Name": atom.GetSymbol(),
                    "Mass": atom.GetMass(),
                    "Charge": atom.GetFormalCharge(),
                    "Element_Name": find_element_index(self.allowable_features['possible_atoms'], str(atom.GetSymbol())),
                    "Hybridization": find_element_index(self.allowable_features['possible_hybridization_list'], str(atom.GetHybridization())),
                    "Num_Hydrogens": atom.GetTotalNumHs(),
                    "Formal_Charge": atom.GetFormalCharge(),
                    "Unpaired_Electron": atom.GetNumRadicalElectrons(),
                    "In_Aromatic_Substructure": int(atom.GetIsAromatic())
                }

                atom_properties = {key: atom_properties[key] for key in properties_to_extract}
                atom_properties_list.append(atom_properties)

        atom_properties_df = pd.DataFrame(atom_properties_list, columns=properties_to_extract)
        return atom_properties_df

In [53]:
# Create an instance of the class
atom_ligand_extractor = AtomPropertiesExtractor()
atom_receptor_extractor =  AtomPropertiesExtractor()

In [54]:
#atom_receptor_extractor.extract_properties(ids_lig['3gv9'][2], node_descriptors, ligand_path.as_posix())

#### Dict with atom properties: Ligand and receptor

In [55]:
#extract_atom_properties(ligand_path.as_posix(), ids_lig['3gv9'][2])

In [56]:
#atom_ligand_extractor.extract_properties(ids_lig['3gv9'][1], config.node_descriptors, '/home/lbcb02/Workspace/Scripts/coreset/3gv9/results/pose_1.pdb')

In [57]:
#ids_lig['3gv9'][2]

In [58]:
lig_atoms_prop = {}
rec_atoms_prop = {}
df_dict = {}

In [59]:
%%time
for pdb in tqdm(pdbs, desc = "Processing PDBs"):
    lig_atoms_prop[pdb] = {}
    rec_atoms_prop[pdb] = {}
    df_dict[pdb] = {}
    print(pdb)
    for pose in range(1, nposes+1):
        ligand_path =   f"{config.coreset}/{pdb}/results/pose_{pose}.pdb"
        protein_path =  f"{config.coreset}/{pdb}/{pdb}_protein.pdb"
       
        lig_atoms_prop[pdb][pose] = atom_ligand_extractor.extract_properties(ids_lig_with_poses[pdb][pose], config.node_descriptors, ligand_path)#.drop(columns = 'Name')
        rec_atoms_prop[pdb][pose] = atom_receptor_extractor.extract_properties(ids_rec_with_poses[pdb][pose], config.node_descriptors, protein_path)#.drop(columns = 'Name')
            
        df_dict[pdb][pose] = df[(df['pdb']==pdb) & (df['pose']==str(pose))]
        #print(rec_atoms_prop[pdb][pose].shape)
        #print(lig_atoms_prop[pdb][pose].shape)

Processing PDBs:   0%|                                   | 0/40 [00:00<?, ?it/s]

2qbq


Processing PDBs:   2%|▋                          | 1/40 [00:00<00:21,  1.85it/s]

1ydr


Processing PDBs:   5%|█▎                         | 2/40 [00:01<00:22,  1.70it/s]

...

1z6e


Processing PDBs:  98%|█████████████████████████▎| 39/40 [00:45<00:01,  1.16s/it]

3bgz


Processing PDBs: 100%|██████████████████████████| 40/40 [00:45<00:00,  1.14s/it]

CPU times: user 45.2 s, sys: 217 ms, total: 45.4 s
Wall time: 45.6 s


In [60]:
df_dict

{'2qbq': {1:      ligand_atom_index ligand_atom_name ligand_chain  ligand_resID  \
  0                  5.0              C13            d           0.0   
  1                  5.0              C13            d           0.0   
  2                  5.0              C13            d           0.0   
  3                  6.0              C14            d           0.0   
  4                  6.0              C14            d           0.0   
  ..                 ...              ...          ...           ...   
  118               27.0              C42            d           0.0   
  119               28.0              C46            d           0.0   
  120               28.0              C46            d           0.0   
  121               28.0              C46            d           0.0   
  122               31.0              C50            d           0.0   
  
      ligand_resName  distance  receptor_atom_index receptor_atom_name  \
  0              MOL  3.847561                46

<font color = red> Should i keep L, R? 

In [47]:
#lig_atoms_prop['Atom_Index'] = "L" + lig_atoms_prop['Atom_Index'].astype(str)
#rec_atoms_prop['Atom_Index'] = "R" + rec_atoms_prop['Atom_Index'].astype(str)

### Preparing data for graph

Obs: For 3gr2 pose 10 we have nothing! 

In [48]:
#torch.tensor(np.array(lig_atoms_prop['3gr2'][10].iloc[:, 1:].astype("float32")))

In [49]:
#torch.tensor(np.array(rec_atoms_prop['3gr2'][pose].iloc[:, 1:].astype("float32")))

<font color ="red ">x_t tensor([], size=(0, 8)) for 3gr2!

#### Label 

Obs: "Error calculating RMSD for 4kz6 pose 1: No sub-structure match found between the reference
and probe mol"
<font color = 'red'> For  now, i will assume the label as 0 to just to begin!

In [52]:
%run Label.ipynb

label.json saved at: ../Datahub/Data


In [53]:
label = pd.read_json(f"{config.data}/label.json").fillna(0)

In [54]:
label.head()

,2qbq,1ydr,3b68,1owh,2vw5,2fxs,2cet,1ydt,2fvd,3ao4,...,1bzc,1bcu,1s38,2yfe,2wvt,2w66,1h23,2xb8,1z6e,3bgz
1,0,1,0,0,1,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,0
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
len(label.columns)

40

In [56]:
# #******************************** PSG-BAR  Altered ********************************
# class Get_dataset(Dataset):
#     def __init__(self):
#        print("init")
#     def get_unique_prot_graphs(self):
#         #calculate protein graphs for all unique pdbs in the data
        
#         self.protgraph_dict = dict()
#         self.failed_protgraph =[]
        
#         for pdb in pdbs:
#             self.protgraph_dict[pdb] = {}  # Initialize an empty dictionary for each pdb
#             for pose in range(1, nposes + 1):
               
#                 x_s = torch.tensor(np.array(lig_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
#                 x_t = torch.tensor(np.array(rec_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
#                 edge_index = torch.tensor(np.array([df_dict[pdb][pose]["ligand_atom_index"].astype("int64"), df_dict[pdb][pose]["receptor_atom_index"].astype("int64")]))
#                 edge_index = map_node_names_to_indices(edge_index)
#                 distances = torch.tensor(np.array(df_dict[pdb][pose]['distance']).astype("float32"))
            
#                 self.protgraph_dict[pdb][pose]= {'x_s':x_s,
#                                                  'x_t':x_t,
#                                                  'edge_index':edge_index,
#                                                  'edge_attr':distances,
#                                                 }

#         return self.protgraph_dict
        
#     def __len__(self):
#         # Return the total number of data samples in your dataset
#         return len(self.protgraph_dict)  # You may need to modify this depending on your data structure

#     def __get__(self, index):
#         pdb = list(self.protgraph_dict.keys())[index]
#         pose = 1  # You may need to modify this
#         data_sample = self.protgraph_dict[pdb][pose]
#         return data_sample

        

In [57]:
class Get_dataset():
    def __init__(self, pdbs, lig_atoms_prop, rec_atoms_prop, df_dict, subsample=None, preprocessed_protgraph=False):
        self.pdbs = pdbs
        self.lig_atoms_prop = lig_atoms_prop
        self.rec_atoms_prop = rec_atoms_prop
        self.df_dict = df_dict
        self.protgraph_dict = self.get_unique_prot_graphs()
        self.preprocessed_protgraph=preprocessed_protgraph
        self.subsample = subsample 
        
    def map_node_names_to_indices(self, edge_index):
        unique_node_names_0, node_name_to_index_0 = torch.unique(edge_index[0], sorted=True, return_inverse=True)
        unique_node_names_1, node_name_to_index_1 = torch.unique(edge_index[1], sorted=True, return_inverse=True)
    
        # Map the node names to their corresponding indices separately for edge_index[0] and edge_index[1]
        updated_edge_index_0 = node_name_to_index_0.view(1, -1)
        updated_edge_index_1 = node_name_to_index_1.view(1, -1)
    
        # Combine the updated edge indices back into the same format as edge_index
        updated_edge_index = torch.cat((updated_edge_index_0, updated_edge_index_1), dim=0)
    
        return updated_edge_index
        
    def get_unique_prot_graphs(self):
        protgraph_dict = {}
        failed_protgraph = []

        for pdb in self.pdbs:
            protgraph_dict[pdb] = {}
            for pose in range(1, nposes + 1):
                x_s = torch.tensor(np.array(self.lig_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
                x_t = torch.tensor(np.array(self.rec_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
                edge_index = torch.tensor(np.array([self.df_dict[pdb][pose]["ligand_atom_index"].astype("int64"),
                                                    self.df_dict[pdb][pose]["receptor_atom_index"].astype("int64")]))
                edge_index = self.map_node_names_to_indices(edge_index)
                distances = torch.tensor(np.array(self.df_dict[pdb][pose]['distance']).astype("float32"))

                protgraph_dict[pdb][pose] = {'x_s': x_s,
                                             'x_t': x_t,
                                             'edge_index': edge_index,
                                             'edge_attr': distances,
                                             }
        return protgraph_dict


    def process(self):
        print('processessing....')
        if self.preprocessed_protgraph:
            with open(f'{config.root}/{config.project_name}/protgraph_dict.pkl','rb') as f:
               protgraph_dict = pickle.load(f)
        else:
            protgraph_dict = self.get_unique_prot_graphs()
            with open(f'{config.root}/{config.project_name}/protgraph_dict.pkl','wb') as f:
                pickle.dump(protgraph_dict,f)
         
        for pdb in pdbs:
            for pose in range(1, nposes + 1): 

                protgraph = self.protgraph_dict[pdb][pose] ### Needs alteration to open saved dict, lets got with it righ now

                interaction_id = str(pdb) + "_" + str(pose)                 
     
                x_s, x_t, edge_index, edge_attr= protgraph['x_s'], protgraph['x_t'], protgraph['edge_index'],protgraph['edge_attr']

                #y = torch.randint(2, (1,), dtype=torch.float32)
                try:
                    y = torch.tensor(label[pdb][pose], dtype=torch.float32)
                except Exception as e:
                    print(f"Error: {e}")
                    continue
                #y = label[pdb][pose].astype("float32")
                data = BipartiteData(edge_index=edge_index, x_s=x_s, x_t=x_t, y=y, edge_attr = edge_attr)
                print(f'saving data {config.processed_dir}')
                torch.save(data, os.path.join(config.processed_dir, f'data_{interaction_id}.pt'))    

    
    def processed_file_names(self):
        datalist =   [torch.load(os.path.join(config.processed_dir, f'{filename}')) for filename in os.listdir(config.processed_dir)]
        return datalist

   # def __len__(self):
   #     return len(protgraph_dict)  ###
   # 
   #def __getitem__(self, index):
   #    pdb = self.pdbs[index]
   #    pose = 1  # You may need to modify this
   #    data_sample = self.protgraph_dict[pdb][pose]
   #    return data_sample

  

dataset = Get_dataset(pdbs, lig_atoms_prop, rec_atoms_prop, df_dict)

In [58]:
#dataset.get_unique_prot_graphs()['3gv9'][1]['edge_index'].shape

#### BipartiteData 

In [59]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None, y=None, edge_attr=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
        self.y = y
        self.edge_attr = edge_attr  # Add edge_attr attribute
        self.num_nodes = (x_s.size(0) if x_s is not None else 0) + (x_t.size(0) if x_t is not None else 0)

    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

#### Dataset

In [60]:
%%time
dataset.process()
dataset_list = dataset.processed_file_names()

processessing....
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datahub/processed_dir
saving data ../Datah

In [61]:
dataset_list

[BipartiteData(edge_index=[2, 0], x_s=[0, 7], x_t=[0, 7], y=0.0, edge_attr=[0], num_nodes=0),
 BipartiteData(edge_index=[2, 148], x_s=[148, 7], x_t=[148, 7], y=0.0, edge_attr=[148], num_nodes=296),
 BipartiteData(edge_index=[2, 101], x_s=[101, 7], x_t=[101, 7], y=0.0, edge_attr=[101], num_nodes=202),
 BipartiteData(edge_index=[2, 110], x_s=[110, 7], x_t=[110, 7], y=0.0, edge_attr=[110], num_nodes=220),
 BipartiteData(edge_index=[2, 120], x_s=[120, 7], x_t=[120, 7], y=0.0, edge_attr=[120], num_nodes=240),
 BipartiteData(edge_index=[2, 167], x_s=[167, 7], x_t=[167, 7], y=0.0, edge_attr=[167], num_nodes=334),
 BipartiteData(edge_index=[2, 85], x_s=[85, 7], x_t=[85, 7], y=0.0, edge_attr=[85], num_nodes=170),
 BipartiteData(edge_index=[2, 0], x_s=[0, 7], x_t=[0, 7], y=0.0, edge_attr=[0], num_nodes=0),
 BipartiteData(edge_index=[2, 149], x_s=[149, 7], x_t=[149, 7], y=0.0, edge_attr=[149], num_nodes=298),
 BipartiteData(edge_index=[2, 151], x_s=[151, 7], x_t=[151, 7], y=0.0, edge_attr=[151], 

In [62]:
len(dataset_list)

400

### Saving pkl 

In [63]:
# Save the list to a file
with open(f'{config.data}/bipartite_data.pkl', 'wb') as file:
    pickle.dump(dataset_list, file)

In [64]:
with open(f'{config.data}/bipartite_data.pkl', 'rb') as file:
    dataset_list = pickle.load(file)

## Dataset info 

In [65]:
len(dataset_list)

400

In [66]:
##### Filter data list 
len(dataset_list)
filtered_data_list_num_nodes = [data for data in dataset_list if data.num_nodes>0]
filtered_data_list_descriptors = [data for data in filtered_data_list_num_nodes if data.x_s.shape[0] > 0 and data.x_t.shape[0] > 0]

In [67]:
filtered_data_list = filtered_data_list_descriptors.copy()

In [68]:
label_distribuition = dict(Counter([label.y.tolist() for label in filtered_data_list]))
amount_of_graphs_used_to_train = len(filtered_data_list)

In [69]:
print(5*"*******")
print("Dataset 2016: 285")
print("Docked complexes: ", len(pdbs))
print("Nº of poses generated: ", len(dataset_list))
#print("Nº graphs with features: ", len(filtered_data_list_num_nodes))
print("Nº Dataset to train ", len(filtered_data_list))
print(5*"*******")

***********************************
Dataset 2016: 285
Docked complexes:  40
Nº of poses generated:  400
Nº Dataset to train  337
***********************************


| Priority | Status      | Task Description                                |
|----------|-------------|-------------------------------------------------|
| 1        | To Do       | -   label                              |
|          |             |   - Problem of rmsd                            |
|          |             |   - Put rmsd method: rdkit                     |
| 1        | To Do       | - Model descriptors                            |
|          |             |   - Nodes meaning                                |
| 2        | To Do       | - Add more metrics                             |
| 2        | To Do       | - Print more metrics                           |
| 3        | To Do       | - Cross-validation method as info              |
| 3        | To Do       | - Revise split methods                         |
| 4        | To Do       | - See what binna considers as close contact    |
| 5        | To Do       | - Clean code                                   |
| 5        | To Do       | - Comment code                                 |
| 5        | To Do       | - Process all data                             |


##### Figure 3: Cumulative fraction of results with crystal-like (within 2 Å RMSD) binding mode plotted versus pose number as ranked

Cumulative fraction of systems that contain at least one correct pose up to a given pose number.

    For example, at x=5, the y-values indicate the fraction of systems that have at least one 
    “positive” pose in the top 5 according to the rankings specified by each model. 
    
    It can be seen that the “L+LP+R” model maintains roughly the advantage seen on the first pose until approximately x=10, where all plots in Figure 3 start to level off. In 65% of systems in our test set, the docking program samples a correct mode in at least one of the 20 rank positions.